In [68]:
import json
import difflib
import copy
from pprint import pprint
import csv
import numpy as np

#Reading the file
f = open('python1007.json')
with open('python1007.json',encoding='utf-8') as f:
    content = f.readlines()
#Getting rid of empty strings
content =  [x.strip() for x in content]
content = list(filter(None, content)) 
TweetsJsonArray = []
#To JSON TYPE
for tweet in content:
    TweetsJsonArray.append(json.loads(tweet))  
#Preprocessing - deleting the similar tweets- deciede that it is not needed
import difflib
def similar(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()

# reading the bad words file 
with open('bad-words.csv') as csvfile:    
    readCSV = csv.reader(csvfile, delimiter=',')
    badwords = []
    weight = []
    count = []
    result = []
    for row in readCSV:       
        badwords.append(row[0])
        weight.append(int(row[1]))

#Calculating the offinsiveness level
def AssesOffensivenessLevel(TweetText):        
        templist = TweetText.split()
        temp = set(badwords).intersection(templist)
        my_list = list(temp)
        print (my_list)
        #print my_list
        final = 0
        result=0
        if len(my_list)!=0:
            for i in my_list:
                index = badwords.index(i)
                final = float(final) + float(weight[index])        
            result = final / len(my_list)
        return result

In [69]:
#Writing the content to the exel for Manual Analysis
import re
import csv
import json
import xlwt
import re

num=1;
book = xlwt.Workbook()
sheet1 = book.add_sheet("Sheet1",cell_overwrite_ok=True)

row = sheet1.row(0)

row.write(0, 'Created at')
row.write(1, 'Text')  
row.write(2, 'Names') 
row.write(3, 'Description') 
row.write(4, 'Locations') 
row.write(5, 'Id') 
row.write(6, 'OffensivenessWeight')
n=0
for tweet in TweetsJsonArray: 
    n=n+1
    if tweet['lang']=="en":
        row = sheet1.row(num)
        row.write(0, tweet['created_at'])  
        text=''
        if 'extended_tweet' in tweet:
            text=tweet['extended_tweet']['full_text']
        elif 'retweeted_status' in tweet:
            if 'extended_tweet' in tweet['retweeted_status']:
                text=tweet['retweeted_status']['extended_tweet']['full_text']
            else:
                text=tweet['retweeted_status']['text']
        else:
            text=tweet['text']
        text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
        names = tweet['user']['name']
        description = tweet['user']['description']
        #text = re.sub(r'\W+', '', text)
        
        PERMITTED_CHARS = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ_- "
        text = "".join(c for c in text if c in PERMITTED_CHARS)
        text=text.lower()
        text=text.strip()
        names="".join(c for c in names if c in PERMITTED_CHARS)
        description="".join(c for c in description if c in PERMITTED_CHARS) if description!= None else ''
        #text=text+";\n"+description       
        num=num+1              
        
        row.write(1, text)
        row.write(2, names)
        row.write(3, description)
        row.write(4, tweet['user']['location'] if tweet['user']['location'] != None else '')
        row.write(5, num)
        num=num+1
book.save("TweetCorpusExel3.xls")
    #csvFile.writerow([tweet["created_at"],tweet["text"],tweet["user"]["name"],tweet["user"]["description"],tweet["user"]["location"]])

In [ ]:
import pandas as pd

xl = pd.ExcelFile("TweetCorpusExel3.xls")
df = xl.parse("Sheet1")
print (df['Text'][2])
df = df.sort_values(by="Text")


writer = pd.ExcelWriter('TweetCorpusExel4.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer.save()
writer.close()

these victims are criminals illegally in our country


In [ ]:
df = pd.read_excel('TweetCorpusExel4.xlsx', sheetname='Sheet1')
writer = pd.ExcelWriter('TweetCorpusExel5.xlsx')
currentrow=0
tweet=df['Text'][currentrow]
l = len(df)
for i in range(l):   
    if df['Text'][i]==tweet or similar(df['Text'][i],tweet)>0.8:
        df['Text'][i]=''        
        df['Id'][currentrow]=str(df['Id'][currentrow])+","+str(df['Id'][i])
        if df['Names'][i]!='':
            df['Names'][currentrow]=str(df['Names'][currentrow]) +","+str(df['Names'][i])
        df['Text'][currentrow]=str(df['Text'][currentrow])+" \n <::> "+str(df['Description'][currentrow])
        df['Locations'][currentrow]=str(df['Locations'][currentrow])+","+str(df['Locations'][i])
        
    else:
        currentrow=i
        tweet=df['Text'][currentrow]
        df['OffensivenessWeight'][currentrow]=AssesOffensivenessLevel(tweet)
df = df[df.Text != '']   
df.to_excel(writer,'Sheet1',index=False)
writer.save()
writer.close()

C:\Users\mmamo_000\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mmamo_000\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mmamo_000\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\mmamo_000\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarni

[]
[]
[]
[]
[]
[]
[]
[]
[]
['jihad', 'gay']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['jihad']
[]
[]
[]
[]
[]
[]
[]
['jihad']
[]
[]
[]
[]
[]
['killer']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['jihad']
['jihad']
['jihad', 'idiot']
['hamas']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['kaffir']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['jihad']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['homo', 'idiot']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['fucked']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['hole']
['fraud']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [1]:
json.dumps(TweetsJsonArray[1]['user'], sort_keys=True, indent=4)

NameError: name 'json' is not defined

In [65]:
AssesOffensivenessLevel("My bad words are not so bad.Ups pee pecker paki")

['paki', 'pee', 'pecker']


2.6666666666666665